# Experiment 05-08

In [13]:
import sys
import numpy as np
import hashlib
from binascii import hexlify
import pyopencl as cl
from Library.opencl_information import opencl_information
import logging
from pqdm.threads import pqdm
from tqdm.notebook import tqdm

## Configure Logging

In [14]:
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
        logging.FileHandler("ex05_08.log"),
        logging.StreamHandler(sys.stdout)
    ])

logging.info("Started")

2022-07-13 14:40:43 INFO     Started


## Show the available Platforms

In [15]:
info = opencl_information()
info.print_full_info()


OpenCL Platforms and Devices
Platform 0 - Name: Apple
Platform 0 - Vendor: Apple
Platform 0 - Version: OpenCL 1.2 (Apr 19 2022 18:44:25)
Platform 0 - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
 Device - Type: ALL | CPU
 Device - Max Clock Speed: 2300 Mhz
 Device - Compute Units: 16
 Device - Local Memory: 32 KB
 Device - Constant Memory: 64 KB
 Device - Global Memory: 16 GB
 Device - Max Buffer/Image Size: 4096 MB
 Device - Max Work Group Size: 1024


 --------------------------------------------------------
 Device - Name: Intel(R) UHD Graphics 630
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1200 Mhz
 Device - Compute Units: 24
 Device - Local Memory: 64 KB
 Device - Constant Memory: 64 KB
 Device - Global Memory: 2 GB
 Device - Max Buffer/Image Size: 384 MB
 Device - Max Work Group Size: 256


 --------------------------------------------------------
 Device - Name: AMD Radeon Pro 

## Configure the OpenCL Context

In [ ]:
platform_number = 0
device_number = 2

cl_devices = cl.get_platforms()[platform_number].get_devices()
cl_ctx = cl.Context(cl_devices)
cl_queue = cl.CommandQueue(cl_ctx, cl_devices[device_number])

: 

: 

In [ ]:
cl_device = cl_devices[device_number]

cl_threads = cl_device.max_compute_units * cl_device.max_work_group_size
if cl_device.type & 4 == 0:
    cl_threads = cl_devices.max_work_group_size

print('Max Compute Units:', cl_device.max_compute_units)
print('Max Compute Units:', cl_device.max_work_group_size)
print('Threads:', cl_threads)

Max Compute Units: 24
Max Compute Units: 256
Threads: 6144


## Compile the Program

In [ ]:
def build_program(program_files : list, cl_ctx : cl.Context,
        build_options=[]) -> cl.Program:
    """
    Build a program from an OpenCL source file.

    Parameters
    ----------
    program_files : list
        The path to the OpenCL source files.
    cl_ctx : pyopencl.Context
        The context to build the program with.
    build_options : list of str
        The build options to use.

    Returns
    -------
    pyopencl.Program
    """
    program_source = ''

    for cl_file in program_files:
        with open(cl_file, 'r') as cl_file:
            file_source = cl_file.read()
            program_source += '\n' + file_source

    program_source = cl.Program(cl_ctx, program_source)
    program = program_source.build(options=build_options)
            
    return program

In [ ]:
cl_program_files = [
    'Library/worker/sha256.cl',
    'Library/worker/zimcoin.cl',
]

cl_program = build_program(cl_program_files, cl_ctx)

# show the kernel names
program_kernel_names = cl_program.get_info(cl.program_info.KERNEL_NAMES)
print(f"Kernel Names: {program_kernel_names}")

Kernel Names: get_single_hash;get_random_string;get_single_hash_nonce;mine_eight;mine_nonce;hash_main;mine_eight_sequential;get_random_numbers


/Users/foulds/env/dsm070/lib/python3.8/site-packages/pyopencl/__init__.py:274: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  warn("Non-empty compiler output encountered. Set the "


## Mine Nonce

In [ ]:
max_nonce = 16
max_output_size = 255

In [ ]:
def display_output(zeros_found : dict, nonce : np.ndarray,
                   nonce_len : np.ndarray, plaintext : str, prefix : str) -> None:
    """
    Display the output generated by the OpenCL Kernel.
    """
    for i in range(0, max_output_size):
        if nonce_len[i] > 0:
            nonce_str = prefix + nonce[i * max_nonce:i * max_nonce + nonce_len[i]].tobytes().decode('UTF-8')
            zeros_found[i] = nonce_str
            hash = hashlib.sha256((plaintext + nonce_str).encode('utf-8'))

            logging.info("%4d: [%2d] %20s %64s" % (i, nonce_len[i], nonce_str, hash.hexdigest() if nonce_len[i] > 0 else ''))

In [ ]:
print(np.iinfo(np.unsignedinteger).max)
print(np.iinfo(np.ulonglong).max)
print(len(str(np.iinfo(np.ulonglong).max)))

18446744073709551615
18446744073709551615
20


In [ ]:
#cl_threads = (cl_threads // 2)
cl_threads = 10

In [ ]:
postfix = ""
base_text = 'this is a description of the latest block'
plaintext = base_text + postfix
plaintext_bytes = np.frombuffer(plaintext.encode('utf-8'), dtype=np.uint8)
plaintext_length = np.int32(len(plaintext_bytes))
cl_plaintext_bytes = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_bytes)
cl_plaintext_length = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=plaintext_length)

# set up the variables to generate the random numbers
seed = np.ulonglong(0)
window_size = np.uint32(1234560)
nonce = np.zeros(shape=max_nonce * max_output_size, dtype=np.uint8)
nonce_len = np.zeros(shape=max_output_size, dtype=np.uint8)
last_nonce = np.zeros(shape=cl_threads, dtype=np.ulonglong)

# allocate the memory for the variables on the device
cl_window_size = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=window_size)
cl_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce.nbytes)
cl_nonce_len = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, nonce_len.nbytes)
cl_last_nonce = cl.Buffer(cl_ctx, cl.mem_flags.WRITE_ONLY, last_nonce.nbytes)

zeros_found = {}

keep_running = True
while (keep_running):
    seed = np.ulonglong(10)
    cl_seed = cl.Buffer(cl_ctx, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=seed)

    # execute the program
    cl_program.mine_eight_sequential(
        cl_queue, (cl_threads,), None,
        cl_seed,
        cl_window_size,
        cl_plaintext_bytes,
        cl_plaintext_length,
        cl_nonce,
        cl_nonce_len,
        cl_last_nonce
    )

    # get the results
    cl.enqueue_copy(cl_queue, nonce, cl_nonce)
    cl.enqueue_copy(cl_queue, nonce_len, cl_nonce_len)
    cl.enqueue_copy(cl_queue, last_nonce, cl_last_nonce)

    # display the last nonce tried for each thread
    # for i in range(0, cl_threads):
    #     logging.info("Thread %d: %d" % (i, last_nonce[i]))

    # display the results
    display_output(zeros_found, nonce, nonce_len, base_text, postfix)
    logging.info('Iteration Complete.')

    keep_running = False

2022-07-13 14:40:23 INFO     build program: kernel 'mine_eight_sequential' was part of a lengthy source build resulting from a binary cache miss (7.22 s)
2022-07-13 14:40:34 INFO        3: [ 2]                      e49fc93861a2c463d4d7fa7858fc52a34f856147d72b98eae2e311612b6e8257
2022-07-13 14:40:34 INFO        4: [ 2]                      e49fc93861a2c463d4d7fa7858fc52a34f856147d72b98eae2e311612b6e8257
2022-07-13 14:40:34 INFO        5: [ 2]                      e49fc93861a2c463d4d7fa7858fc52a34f856147d72b98eae2e311612b6e8257
2022-07-13 14:40:34 INFO        6: [ 2]                      e49fc93861a2c463d4d7fa7858fc52a34f856147d72b98eae2e311612b6e8257
2022-07-13 14:40:34 INFO        7: [ 2]                      e49fc93861a2c463d4d7fa7858fc52a34f856147d72b98eae2e311612b6e8257
2022-07-13 14:40:34 INFO        8: [ 2]                      e49fc93861a2c463d4d7fa7858fc52a34f856147d72b98eae2e311612b6e8257
2022-07-13 14:40:34 INFO        9: [ 2]                      e49fc93861a2c463d4d7fa7858fc5

In [12]:
input_buffer = {}
nonce = {}
nonce_len = 0
seed = 123456

while (seed > 0):
    nonce_len += 1
    input_buffer[nonce_len - 1] = (seed % 10)
    seed = seed // 10

print(input_buffer)

for i in range(0, nonce_len):
    nonce[nonce_len-1 - i] = input_buffer[i]

for i in range(0, nonce_len):
    print(nonce[i])
    

{0: 6, 1: 5, 2: 4, 3: 3, 4: 2, 5: 1}
{5: 6, 4: 5, 3: 4, 2: 3, 1: 2, 0: 1}
